<a href="https://colab.research.google.com/github/ajai-cr7/DiskFailurePredictionDeepLearning/blob/main/notebookb2014_10_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# load the data set
data = pd.read_csv('/content/2014-10-23.csv')

# print info about columns in the dataframe
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38592 entries, 0 to 38591
Data columns (total 85 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  38592 non-null  object 
 1   serial_number         38592 non-null  object 
 2   model                 38592 non-null  object 
 3   capacity_bytes        38592 non-null  int64  
 4   failure               38592 non-null  int64  
 5   smart_1_normalized    38592 non-null  int64  
 6   smart_1_raw           38592 non-null  int64  
 7   smart_2_normalized    22335 non-null  float64
 8   smart_2_raw           22335 non-null  float64
 9   smart_3_normalized    38592 non-null  int64  
 10  smart_3_raw           38592 non-null  int64  
 11  smart_4_normalized    38592 non-null  int64  
 12  smart_4_raw           38592 non-null  int64  
 13  smart_5_normalized    38592 non-null  int64  
 14  smart_5_raw           38592 non-null  int64  
 15  smart_7_normalized 

In [4]:
# normalise the amount column
# data['normAmount'] = StandardScaler().fit_transform(np.array(data['Amount']).reshape(-1, 1))

# drop Time and Amount columns as they are not relevant for prediction purpose
data = data.drop(['date', 'serial_number','model','capacity_bytes'], axis = 1)

data['failure'].value_counts()
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38592 entries, 0 to 38591
Data columns (total 81 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   failure               38592 non-null  int64  
 1   smart_1_normalized    38592 non-null  int64  
 2   smart_1_raw           38592 non-null  int64  
 3   smart_2_normalized    22335 non-null  float64
 4   smart_2_raw           22335 non-null  float64
 5   smart_3_normalized    38592 non-null  int64  
 6   smart_3_raw           38592 non-null  int64  
 7   smart_4_normalized    38592 non-null  int64  
 8   smart_4_raw           38592 non-null  int64  
 9   smart_5_normalized    38592 non-null  int64  
 10  smart_5_raw           38592 non-null  int64  
 11  smart_7_normalized    38592 non-null  int64  
 12  smart_7_raw           38592 non-null  int64  
 13  smart_8_normalized    22338 non-null  float64
 14  smart_8_raw           22338 non-null  float64
 15  smart_9_normalized 

In [5]:
data.fillna(data.mean(), inplace=True)

# data = data[["failure","smart_1_normalized","smart_2_normalized","smart_3_normalized","smart_4_normalized"]]

#take first 100000 rows with 5 column attributes
data = data.loc[1:38000, ["smart_1_normalized","smart_3_normalized","smart_4_normalized","smart_5_normalized","failure"]]

#change float to int
data["smart_1_normalized"] = data["smart_1_normalized"].fillna(0.0).astype(int)
data["smart_3_normalized"] = data["smart_3_normalized"].fillna(0.0).astype(int)
data["smart_4_normalized"] = data["smart_4_normalized"].fillna(0.0).astype(int)
data["smart_5_normalized"] = data["smart_5_normalized"].fillna(0.0).astype(int)

print(data)

#print how many nan values present 
# print(data["failure"].isnull().sum())

#display how much data is failure or success
data['failure'].value_counts()

       smart_1_normalized  smart_3_normalized  ...  smart_5_normalized  failure
1                     100                 127  ...                 100        0
2                     100                 124  ...                 100        0
3                     100                 137  ...                 100        0
4                     200                 195  ...                 200        0
5                     100                 131  ...                 100        0
...                   ...                 ...  ...                 ...      ...
37996                 118                  97  ...                 100        0
37997                 100                 124  ...                 100        0
37998                 100                 126  ...                 100        0
37999                 100                 100  ...                 100        0
38000                 105                  94  ...                 100        0

[38000 rows x 5 columns]


0    37992
1        8
Name: failure, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X = data[["smart_1_normalized","smart_3_normalized","smart_4_normalized","smart_5_normalized"]]
y=data["failure"]

# split into 70:30 ration
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# describes info about train and test set
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (26600, 4)
Number transactions y_train dataset:  (26600,)
Number transactions X_test dataset:  (11400, 4)
Number transactions y_test dataset:  (11400,)


In [10]:
# print(data)

# print(data.info())
import warnings
warnings.filterwarnings("ignore")
# logistic regression object
lr = LogisticRegression()

# train the model on train set
lr.fit(X_train, y_train.ravel())

predictions = lr.predict(X_test)

# print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11399
           1       0.00      0.00      0.00         1

    accuracy                           1.00     11400
   macro avg       0.50      0.50      0.50     11400
weighted avg       1.00      1.00      1.00     11400



In [11]:
import warnings
warnings.filterwarnings("ignore")

print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))

# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

Before OverSampling, counts of label '1': 7
Before OverSampling, counts of label '0': 26593 

After OverSampling, the shape of train_X: (53186, 4)
After OverSampling, the shape of train_y: (53186,) 

After OverSampling, counts of label '1': 26593
After OverSampling, counts of label '0': 26593


In [9]:
lr1 = LogisticRegression()
lr1.fit(X_train_res, y_train_res.ravel())
predictions = lr1.predict(X_test)

# print classification report
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       1.00      0.50      0.67     11399
           1       0.00      1.00      0.00         1

    accuracy                           0.50     11400
   macro avg       0.50      0.75      0.33     11400
weighted avg       1.00      0.50      0.67     11400

